# Code to generate Packet Dataset

In [6]:
import scapy.all as scapy
from scapy.layers.inet import IP, TCP, UDP
import torch
import numpy as np
from collections import deque
import time
import subprocess
import datetime

# Constants
SEQUENCE_LENGTH = 5
NUM_FEATURES = 10
CAPTURE_DURATION = None  # Capture for 3 hrs
def extract_features(packet):
    features = [0] * NUM_FEATURES

    if IP in packet:
        features[0] = len(packet)
        features[1] = packet[IP].tos
        features[2] = packet[IP].ttl
        features[3] = packet[IP].proto
        
        if TCP in packet:
            features[4] = packet[TCP].sport
            features[5] = packet[TCP].dport
            features[6] = packet[TCP].window
            features[7] = len(packet[TCP].payload)
        elif UDP in packet:
            features[4] = packet[UDP].sport
            features[5] = packet[UDP].dport
            features[7] = len(packet[UDP].payload)

        src_ip = packet[IP].src
        dst_ip = packet[IP].dst
        features[8] = int(src_ip.split('.')[-1])  # Last octet of source IP
        features[9] = int(dst_ip.split('.')[-1])  # Last octet of destination IP

    return features

def classify_packet(packet):
    if IP in packet:
        if TCP in packet:
            dport = packet[TCP].dport
            sport = packet[TCP].sport
        elif UDP in packet:
            dport = packet[UDP].dport
            sport = packet[UDP].sport
        else:
            return 'Normal'  # Neither TCP nor UDP

        if dport == 53 or sport == 53:  # DNS
            return 'Real Time'
        elif dport in [80, 443]:  # HTTP/HTTPS
            return 'Web download'
        elif dport in [3074, 3075]:  # Xbox Live
            return 'Games'
        elif dport in [1935, 1936, 5222]:  # RTMP, XMPP
            return 'Streaming'
        elif dport == 22:  # SSH
            return 'Real Time'
        elif dport in [123]:  # NTP
            return 'Real Time'
    
    return 'Normal'  # Default classification
def get_wifi_interface():
    try:
        # Run netsh command to get WiFi interface information
        result = subprocess.run(["netsh", "wlan", "show", "interfaces"], capture_output=True, text=True)
        output = result.stdout

        # Parse the output to find the name of the connected WiFi interface
        for line in output.split('\n'):
            if "Name" in line:
                return line.split(':')[1].strip()
    except Exception as e:
        print(f"Error getting WiFi interface: {e}")
    return None

def capture_packets(interface):
    packet_buffer = deque(maxlen=SEQUENCE_LENGTH)
    features_list = []
    labels = []
    start_time = time.time()

    def packet_callback(packet):
        nonlocal features_list, labels

        features = extract_features(packet)
        packet_buffer.append(features)

        if len(packet_buffer) == SEQUENCE_LENGTH:
            features_list.append(list(packet_buffer))
            labels.append(classify_packet(packet))

        if time.time() - start_time > CAPTURE_DURATION:
            return True  # Stop capture

    print(f"Capturing packets on {interface} for {CAPTURE_DURATION} seconds...")
    scapy.sniff(iface=interface, prn=packet_callback, store=False, stop_filter=lambda p: packet_callback(p))

    return np.array(features_list), labels

def main():

    global CAPTURE_DURATION
    print("5 min = 300 sec\n10min = 600 sec\n1hr = 3600sec\n")
    CAPTURE_DURATION = int(input("Enter Capture Duration in seconds:"))

    interface = get_wifi_interface()
    if not interface:
        print("Could not find WiFi interface. Please check your network connections.")
        return

    print(f"Using WiFi interface: {interface}")
    
    X, y = capture_packets(interface)
    
    # Convert to PyTorch tensors
    X_tensor = torch.FloatTensor(X)
    y_list = y  # Keep y as a list of strings
    

     # Generate timestamp
    timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")

    # Save the dataset
    torch.save((X_tensor, y_list), f'packet_dataset_{timestamp}_{CAPTURE_DURATION}.pt')
    
    print(f"Dataset saved. Shape: {X_tensor.shape}, Labels: {len(y_list)}")

if __name__ == "__main__":
    main()

ModuleNotFoundError: No module named 'torch'

# train new prioritizer model

In [11]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import datetime
from os.path import isfile
from sys import argv
from torch.cuda.amp import GradScaler
from torch.amp import autocast



class PacketPrioritizer(nn.Module):
    def __init__(self):
        super().__init__()
        self.lstm = nn.LSTM(input_size=10, hidden_size=64, num_layers=2, batch_first=True)
        self.fc1 = nn.Linear(64, 32)
        self.fc2 = nn.Linear(32, 6)  # 6 output classes for 6 priority levels
        self.relu = nn.ReLU()

    def forward(self, x):
        lstm_out, _ = self.lstm(x)
        x = self.relu(self.fc1(lstm_out[:, -1, :]))
        return self.fc2(x)

def train_model(model, train_loader, device, num_epochs=50):
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters())
    scaler = GradScaler()
    
    for epoch in range(num_epochs):
        model.train()
        total_loss = 0
        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device, non_blocking=True), labels.to(device, non_blocking=True)
            optimizer.zero_grad()
            
            with autocast("cuda"):
                outputs = model(inputs)
                loss = criterion(outputs, labels)
            
            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()
            
            total_loss += loss.item()
        
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {total_loss/len(train_loader):.4f}')

def prepare_data(X, y):
    # Convert priorities to class indices (0-5)
    priority_to_class = {
        'Games': 0,
        'Real Time': 1,
        'Streaming': 2,
        'Normal': 3,
        'Web download': 4,
        'App download': 5
    }
    y_classes = torch.tensor([priority_to_class[p] for p in y])
    
    # Create DataLoader
    dataset = TensorDataset(X, y_classes)
    return DataLoader(dataset, batch_size=128, shuffle=True, pin_memory=True, num_workers=4)


def main(datasetName):
    
    if datasetName == None:
        while True:
            datasetName = input("Enter the dataset (.pt) file name:")
            if isfile(datasetName) == False:
                print(f"File doesnt exist.{datasetName} Retry.")
            else:
                break

    # Check if CUDA is available
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"Using device: {device}")

    X, y = torch.load(datasetName, weights_only=True)
    
    # Prepare data
    train_loader = prepare_data(X, y)
    
    # Initialize the model and move it to GPU
    model = PacketPrioritizer().to(device)
    
    # Train the model
    train_model(model, train_loader, device)
    
      
    # Generate timestamp
    timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")

    # updated PTH model's filename
    new_PTH_FILENAME = f"packet_prioritizer_{timestamp}.pth"


    # Save the trained model
    torch.save(model.state_dict(), new_PTH_FILENAME)
    
    print("Model trained and saved successfully.")

if __name__ == "__main__":
    file = argv[1] if len(argv) > 1 else None
    main(file)

ModuleNotFoundError: No module named 'torch'

# retrain existing prioritixer model

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from os.path import isfile
import datetime
from sys import argv

# globals
PTH_FILENAME = None
newDatasetName = None

class PacketPrioritizer(nn.Module):
    def __init__(self):
        super().__init__()
        self.lstm = nn.LSTM(input_size=10, hidden_size=64, num_layers=2, batch_first=True)
        self.fc1 = nn.Linear(64, 32)
        self.fc2 = nn.Linear(32, 6)  # 6 output classes for 6 priority levels
        self.relu = nn.ReLU()

    def forward(self, x):
        lstm_out, _ = self.lstm(x)
        x = self.relu(self.fc1(lstm_out[:, -1, :]))
        return self.fc2(x)

def train_model(model, train_loader, device, num_epochs=50):
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters())
    
    for epoch in range(num_epochs):
        model.train()
        total_loss = 0
        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
        
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {total_loss/len(train_loader):.4f}')

def prepare_data(X, y):
    priority_to_class = {
        'Games': 0,
        'Real Time': 1,
        'Streaming': 2,
        'Normal': 3,
        'Web download': 4,
        'App download': 5
    }
    y_classes = torch.tensor([priority_to_class[p] for p in y])
    
    dataset = TensorDataset(X, y_classes)
    return DataLoader(dataset, batch_size=32, shuffle=True)

def main(pth_file, datasetName):
    global PTH_FILENAME, newDatasetName
    if pth_file == None and datasetName == None:
        while True:
            if pth_file == None:
                PTH_FILENAME = input("Enter the previously trained model (.pth) file name:")
            if datasetName == None:
                newDatasetName = input("Enter the training dataset (.pt) file name:")
            if isfile(PTH_FILENAME) == True and isfile(newDatasetName) == True:
                break
            else:
                print("File(s) doesnt exist. Retry.")
    

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"Using device: {device}")

    # Load the pre-trained model
    model = PacketPrioritizer()
    model.load_state_dict(torch.load(PTH_FILENAME))
    model.to(device)
    print("Pre-trained model loaded successfully.")

    # Load the new dataset
    X, y = torch.load(newDatasetName)
    
    # Prepare data
    train_loader = prepare_data(X, y)
    
    # Train the model again
    print("Starting additional training...")
    train_model(model, train_loader, device)
    
    # Generate timestamp
    timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")

    # updated PTH model's filename
    new_PTH_FILENAME = f"packet_prioritizer_{timestamp}.pth"

    # Save the updated model
    torch.save(model.state_dict(), new_PTH_FILENAME)
    
    print(f"Model retrained and saved successfully as {new_PTH_FILENAME}")

if __name__ == "__main__":
    file1,file2 = None, None if len(argv) < 2 else argv[1], argv[2]
    main(file1, file2)